Below is an outline of how you can set up the problem mathematically and solve it using dynamic programming.

---

### 1. **Formulate the Problem as an MDP**

We view each turn of the Yacht dice game as a finite‐horizon Markov Decision Process (MDP). In an MDP you need:

- **States:** A state must encode everything you need to make future decisions. In this case, a state can be defined as
  $$
  s = (\text{scorecard}, d, r, t),
  $$
  where
  - **scorecard** tells which categories have been used and their scores,
  - $d$ is the current outcome of the 5 dice,
  - $r$ is the number of remaining rerolls (0, 1, 2, or 3),
  - $t$ is the number of turns (or categories) remaining (initially 12).

- **Actions:** In any state there are two types of actions:
  1. **Score:** Choose one available category $i$. The immediate reward is given by a scoring function $f_i(d)$ that evaluates the dice $d$ for that category.
  2. **Reroll:** Choose a subset $R \subseteq \{1,\ldots,5\}$ of dice to reroll (if $r>0$).

- **Transition Probabilities:**  
  - If you **score** a category $i$ in state $s$, you get an immediate reward $f_i(d)$, and the game moves to a new turn. At the start of a new turn you typically have a fresh dice roll (or you treat the expected value of a “fresh roll” as the value of the new state with $r=3$ and $t-1$).
  - If you **reroll** a chosen subset $R$, each die in $R$ is replaced by a random number uniformly drawn from $\{1,2,\ldots,6\}$. The new state becomes $s'=(\text{scorecard}, d', r-1, t)$, where $d'$ has the new outcomes in positions $R$ and unchanged outcomes for the other dice.

- **Reward:** Immediate reward comes only when you choose to score a category. Otherwise, rerolling gives no immediate reward but potentially improves future outcomes.

---

### 2. **The Value Function and Bellman Equations**

Let $V(s)$ denote the maximum expected total score achievable from state $s$. We now write the Bellman equations that capture the optimal decision at each state.

#### **Terminal Condition**

When there are no turns left ($t=0$), the game is over:
$$
V(s) = 0.
$$

#### **Scoring Action**

If you decide to score in the current state $s = (\text{scorecard}, d, r, t)$, then for each available category $i$ you have an immediate reward $f_i(d)$ and the game transitions to a new turn. Let
$$
V_{\text{score}}(s) = \max_{i \in \text{Avail}(\text{scorecard})} \left\{ f_i(d) + \mathbb{E}_{d'}\Big[V(\text{scorecard}\cup\{i\}, d', 3, t-1)\Big] \right\}.
$$
Here, the expectation $\mathbb{E}_{d'}$ is over the initial roll (or distribution of dice) for the next turn.

#### **Reroll Action**

If you have at least one reroll left ($r > 0$), you can choose a subset $R$ to reroll. For a given choice of $R$, the new dice outcome is random:
$$
V_{\text{reroll}}(s) = \max_{R \subseteq \{1,\dots,5\}} \ \mathbb{E}_{x \sim \text{Uniform}(1\ldots6)^{|R|}}\left[ V\Big(\text{scorecard}, d_{-R} \cup x, r-1, t \Big) \right],
$$
where $d_{-R}$ denotes the dice not chosen to be rerolled and $x$ are the new outcomes for the positions in $R$.

#### **Combined Bellman Equation**

Thus, the Bellman equation for a state $s = (\text{scorecard}, d, r, t)$ is:
$$
V(s) =
\begin{cases}
\displaystyle V_{\text{score}}(s) & \text{if } r=0, \\
\displaystyle \max\Big\{V_{\text{score}}(s),\; V_{\text{reroll}}(s)\Big\} & \text{if } r > 0.
\end{cases}
$$

In words, if no rerolls remain, you must score; if rerolls remain, you choose the action (scoring immediately or rerolling some dice) that gives the highest expected total score.

---

### 3. **Solving the Dynamic Program**

Because the game has a finite horizon (12 turns) and the state space is (in principle) finite (though large), you can solve the DP using **backward induction**:

1. **Base Case:** For all states with $t = 0$, set $V(s) = 0$.

2. **Backward Recursion:** For $t=1,2,\dots,12$ and for all possible dice outcomes and reroll counts $r$:
   - Compute $V_{\text{score}}(s)$ for each state.
   - If $r > 0$, also compute $V_{\text{reroll}}(s)$ by evaluating the expectation over all possible outcomes for every possible subset $R$.
   - Set
     $$
     V(s) =
     \begin{cases}
     V_{\text{score}}(s), & \text{if } r=0, \\
     \max\{V_{\text{score}}(s), V_{\text{reroll}}(s)\}, & \text{if } r>0.
     \end{cases}
     $$

3. **Policy Extraction:** The optimal decision in each state is the action (either a specific reroll decision or a scoring category) that attains the maximum in the Bellman equation.

---

### 4. **Mathematical Tools and Considerations**

- **Expectation Computation:** For each reroll action, you must sum over all possible outcomes of the dice being rerolled. If $k = |R|$ dice are rerolled, there are $6^k$ outcomes, each with probability $1/6^k$.

- **State Space Reduction:** Due to symmetry in dice outcomes (order does not matter) and possibly similar scorecard patterns, you may use state aggregation to reduce the computational load.

- **Curse of Dimensionality:** Even with aggregation, the state space is large. Efficient computation might require pruning non-promising actions or using approximate dynamic programming methods.

- **Finite Horizon:** Since the game lasts exactly 12 turns, the backward induction is done over these 12 stages. The horizon ensures that you can eventually compute $V(s)$ from the terminal condition.

---

### 5. **Summary of the Process**

1. **Define the state $s=(\text{scorecard}, d, r, t)$ and action space (score or reroll with subset $R$).**

2. **Determine transition probabilities and rewards:**
   - Scoring yields $f_i(d)$ plus transition to a new turn.
   - Rerolling leads to a distribution of new dice outcomes and reduces $r$.

3. **Write the Bellman equation:**
   $$
   V(s) = \begin{cases}
   \displaystyle \max_{i \in \text{Avail}(\text{scorecard})}\left\{f_i(d) + \mathbb{E}_{d'}[V(\text{scorecard}\cup\{i\}, d', 3, t-1)]\right\}, & r=0, \\
   \displaystyle \max\Biggl\{ \max_{i \in \text{Avail}(\text{scorecard})}\left\{f_i(d) + \mathbb{E}_{d'}[V(\text{scorecard}\cup\{i\}, d', 3, t-1)]\right\}, \\
   \hspace{3cm} \max_{R\subseteq\{1,\dots,5\}}\mathbb{E}\left[V(\text{scorecard}, d_{-R}\cup x, r-1, t)\right] \Biggr\}, & r>0.
   \end{cases}
   $$

4. **Solve via backward induction starting at $t=0$ (terminal states) and moving backward to $t=12$.**

5. **Extract the optimal decision policy** by recording which action (scoring category or specific reroll subset) gives the maximum value at each state.

---

This framework provides the mathematical foundation and process for determining the optimal decision at every turn in the Yacht dice game using dynamic programming.

# All availbale categories in Scorecard

- Ones, twos, ... , Sixes : total 6 category. Score : the number of corresponding category's number
  - if sum >= 63, bonus 35 points.
- Choice : Score: Sum of dice
- Full House : Score: Sum of Dice, 1 triple and 1 pair
- 4 of a kind : Score : sum of dice, 4 numbers in common
- Large straight: 12345, or 23456, Score : 30 points
- small straight : 1234, 2345, 3456, Score: 15 points
- Yacht: all same number, Score : 50

  Total : 12 categories, bonus point

In [1]:
category = ["ones", "twos","threes", "fours", "fives","sixes",
            "choice", "full house", "4 of a kind",
            "small straight", "large straight",
            "Yacht"]
for i, name in enumerate(category):
    print(f"Category {name}'s index : {i}")


Category ones's index : 0
Category twos's index : 1
Category threes's index : 2
Category fours's index : 3
Category fives's index : 4
Category sixes's index : 5
Category choice's index : 6
Category full house's index : 7
Category 4 of a kind's index : 8
Category small straight's index : 9
Category large straight's index : 10
Category Yacht's index : 11


In [14]:
import random
import itertools
from functools import lru_cache
from collections import Counter

## State : (Score, dice result, remaining rerolls, available categories)

class DiceState:
    def __init__(self, dice_result=[0, 0, 0, 0, 0]) -> None:
        # All possible dice outcomes (order does not matter)
        self.allDiceState = list(itertools.combinations_with_replacement(range(1, 7), 5))
        pmf = [120, 600, 300, 200, 50, 25, 1]
        self.pmf = tuple(x / 1296 for x in pmf)
        # Use indices 0..4 for 5 dice
        self.allHoldSet = DiceState.get_all_subsets(set(range(5)))
        self.diceResult = dice_result

    @staticmethod
    def get_all_subsets(my_set):
        """Generates all subsets of a given set, including the empty set."""
        subsets = []
        for i in range(len(my_set) + 1):
            for subset_tuple in itertools.combinations(my_set, i):
                subsets.append(set(subset_tuple))
        return subsets

    def prob_dice(self, dice_result):
        """
        Returns the probability of getting dice_result when rolling all 5 dice.
        """
        dice_counts = Counter(dice_result)
        multiplicity = list(dice_counts.values())
        if 5 in multiplicity:
            return self.pmf[6]
        elif 4 in multiplicity:
            return self.pmf[5]
        elif 3 in multiplicity and 2 in multiplicity:
            return self.pmf[4]
        elif 3 in multiplicity:
            return self.pmf[3]
        elif 2 in multiplicity:
            ones = sum(1 for x in multiplicity if x == 1)
            if ones == 1:
                return self.pmf[2]
            else:
                return self.pmf[1]
        else:
            return self.pmf[0]

    @staticmethod
    def prob_transition(A, B, hold):
        """
        Returns the probability of transitioning from dice outcome A to B
        when holding the dice at positions in 'hold'.
        A and B are sequences (or tuples) representing dice results.
        """
        k = 5 - len(hold)  # positions not held will be rerolled.
        held_values = {A[i] for i in hold}
        if not held_values.issubset(set(B)):
            return 0
        return 1 / (6 ** k)

    def get_possible_diceresult(self, n: int):
        return list(itertools.product(range(1, 7), repeat=n))

# ------------------------------
# Helper functions for scoring and DP
# ------------------------------

def immediate_score(category, dice_result):
    """Computes the score for a given category based on dice_result."""
    dice_counts = {i: dice_result.count(i) for i in range(1, 7)}
    if category == 0:  # Ones
        return dice_counts.get(1, 0) * 1
    if category == 1:  # Twos
        return dice_counts.get(2, 0) * 2
    if category == 2:  # Threes
        return dice_counts.get(3, 0) * 3
    if category == 3:  # Fours
        return dice_counts.get(4, 0) * 4
    if category == 4:  # Fives
        return dice_counts.get(5, 0) * 5
    if category == 5:  # Sixes
        return dice_counts.get(6, 0) * 6
    if category == 6:  # Full House
        return 25 if (2 in dice_counts.values() and 3 in dice_counts.values()) else 0
    if category == 7:  # Four of a Kind
        return sum(dice_result) if max(dice_counts.values()) >= 4 else 0
    if category == 8:  # Small Straight
        return 30 if any(set(seq).issubset(dice_result) for seq in [(1,2,3,4), (2,3,4,5), (3,4,5,6)]) else 0
    if category == 9:  # Large Straight
        return 40 if set(dice_result) in [{1,2,3,4,5}, {2,3,4,5,6}] else 0
    if category == 10:  # Yacht
        return 50 if len(set(dice_result)) == 1 else 0
    if category == 11:  # Chance
        return sum(dice_result)
    return 0

def return_remain_category(S: list):
    """Returns the indices of categories that have not been filled (score is 0)."""
    return sorted(i for i, score in enumerate(S) if score == 0)

def filling_scorecard(S: list, d: list, cat: int, a: list, val: int):
    """
    Returns a tuple (newS, newa) where:
      - newS: scorecard after filling the given category with the value.
      - newa: available categories after removing the filled category.
    """
    newS = S.copy()
    newa = a.copy()
    newS[cat] = val
    newa.remove(cat)  # remove cat because it is now filled
    return newS, sorted(newa)

def expected_over_initialroll(S: list, r:int, a: list, cat: int):
    """
    Calculate the weighted sum (expected value) for the next turn’s state
    after scoring a category.

    ## Revised
    * reason : Too many time is consumed... 
    * description changed to : Calculate the future value of initial state, with partially filled scorecard.
    """
    """Old version
    total = 0
    dice_state = DiceState()
    for outcome in dice_state.allDiceState:
        prob = dice_state.prob_dice(outcome)
        av = a.copy()
        if cat in av:
            av.remove(cat)
        # Convert new state components to tuples when calling value_for_state.
        total += prob * value_for_state(tuple(S), tuple(outcome), r, tuple(av))[1]
    """
    # Revised
    upperpoint = sum(S[0:6])
    return total

def expected_over_reroll(S: list, d: list, r: int, a: list, h: frozenset):
    """
    Calculate the weighted sum of state values over all outcomes after a reroll.
      Note: For caching purposes, we convert the hold set to a frozenset.
    """
    dice = DiceState()
    total = 0
    for outcome in dice.allDiceState:
        prob = DiceState.prob_transition(d, outcome, h)
        if prob == 0:
            continue
        total += prob * value_for_state(tuple(S), tuple(outcome), r - 1, tuple(a))[1]
    return total

# ------------------------------
# Dynamic Programming via Cached Value Function
# ------------------------------

@lru_cache(maxsize=None)
def value_for_state(S, d, r, a):
    """
    Returns a tuple (action, expected value) for state:
      S : scorecard (tuple of 12 numbers)
      d : dice result (tuple of length 5)
      r : remaining rerolls (int)
      a : available categories (tuple of indices)
    """
    S_list = list(S)
    d_list = list(d)
    a_list = list(a)
    if a_list:
        print(f"a_list is now {a_list}")

    # Terminal condition: game over
    if not a_list:
        return (None, 0)

    # Base case: if no rerolls remain, must score.
    if r == 0:
        return value_for_scoreoption(tuple(S_list), tuple(d_list), r, tuple(a_list))
    
    score_option = value_for_scoreoption(tuple(S_list), tuple(d_list), r, tuple(a_list))
    reroll_option = value_for_rerolloption(tuple(S_list), tuple(d_list), r, tuple(a_list))

    print(f"score_option is {score_option} and reroll_option is {reroll_option}")
    
    if score_option[1] > reroll_option[1]:
        print("Optimal choice was decided as scoring")
        return score_option
    else:
        return reroll_option

@lru_cache(maxsize=None)
def value_for_scoreoption(S: tuple, dice_result: tuple, r: int, a: tuple, gamma=0.95):
    """
    Evaluate the immediate scoring options.
    Returns a tuple (best_category, expected total value) for scoring.
    """
    print(f"Calculating scoring option's value with : score({S}), dice({dice_result}), reroll({r}), available cat to filled :{a}")
    if not a:
        return (None,0)
    S_list = list(S)
    dice_list = list(dice_result)
    a_list = list(a)
    
    best_value = float('-inf')
    best_cat = None
    for cat in a_list:
        immediate = immediate_score(cat, dice_list)
        print(f"immediate score for {cat} is {immediate}")
        newS, newA = filling_scorecard(S_list, dice_list, cat, a_list, immediate)
        future = gamma * expected_over_initialroll(newS, 3, newA, cat)
        print(f"future value is {future} if scored at {cat} with value {immediate}")
        total_val = immediate + future
        if total_val > best_value:
            best_value = total_val
            best_cat = cat
    print(f"Best choice under scoring was: {best_value} at {best_cat}")
    return (best_cat, best_value)

@lru_cache(maxsize=None)
def value_for_rerolloption(S: tuple, dice_result: tuple, r: int, a: tuple, gamma=0.95):
    """
    Evaluate the reroll option (selecting which dice to hold).
    Returns a tuple (best_hold_set, expected total value) for rerolling.
    """
    print(f"Calculating reroll option's value with : score({S}), dice({dice_result}), reroll({r}), available cat to filled :{a}")
    if not a:
        return (None,0)
    S_list = list(S)
    dice_list = list(dice_result)
    a_list = list(a)
    
    dice = DiceState(dice_list)
    best_value = float('-inf')
    best_hold = None
    # To make the hold set hashable, we use frozenset.
    for hold in dice.allHoldSet:
        hold_fs = frozenset(hold)
        ev = expected_over_reroll(S_list, dice_list, r, a_list, hold_fs)
        if ev > best_value:
            best_value = ev
            best_hold = hold
    print(f"Best choice under reroll was holding {best_hold} with future value {best_value}")
    return (best_hold, gamma * best_value)

# ------------------------------
# Example Simulation
# ------------------------------

if __name__ == "__main__":
    categories = ["ones", "twos", "threes", "fours", "fives", "sixes",
                  "full house", "4 of a kind",
                  "small straight", "large straight",
                  "Yacht", "chance"]
    score = [2, 2, 3, 0, 5, 6, 15, 0, 15, 30, 50, 17]
    available = return_remain_category(score)
    dice_result = [1, 1, 1, 2, 3]
    # Pass state as tuples so that they are hashable.
    decision = value_for_state(tuple(score), tuple(dice_result), 1, tuple(available))
    print("Dice:", dice_result)
    print("Optimal decision (action, expected value):", decision)


a_list is now [3, 7]
Calculating scoring option's value with : score((2, 2, 3, 0, 5, 6, 15, 0, 15, 30, 50, 17)), dice((1, 1, 1, 2, 3)), reroll(1), available cat to filled :(3, 7)
immediate score for 3 is 0
a_list is now [7]
Calculating scoring option's value with : score((2, 2, 3, 0, 5, 6, 15, 0, 15, 30, 50, 17)), dice((1, 1, 1, 1, 1)), reroll(3), available cat to filled :(7,)
immediate score for 7 is 5
future value is 0.0 if scored at 7 with value 5
Best choice under scoring was: 5.0 at 7
Calculating reroll option's value with : score((2, 2, 3, 0, 5, 6, 15, 0, 15, 30, 50, 17)), dice((1, 1, 1, 1, 1)), reroll(3), available cat to filled :(7,)
a_list is now [7]
Calculating scoring option's value with : score((2, 2, 3, 0, 5, 6, 15, 0, 15, 30, 50, 17)), dice((1, 1, 1, 1, 1)), reroll(2), available cat to filled :(7,)
immediate score for 7 is 5
future value is 0.0 if scored at 7 with value 5
Best choice under scoring was: 5.0 at 7
Calculating reroll option's value with : score((2, 2, 3, 0, 5

KeyboardInterrupt: 

## The Problem
1. State space is too large.
2. Too long time convergence
   
## Solution suggest
### 1. grouping states, and using heuristics
When we decide any decision, one can choose
1. maximizing the number of identicals. (-> 4 of a kind, Yacht)
2. maximizing the number of certain number (-> ones, twos, ..., sixes)
3. 